In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

import statistics
from statistics import mode

from nltk import ngrams

import warnings
warnings.filterwarnings('ignore')

# Function on the app

## Hotel Search 

Penjelasan Function:
Function Hotel Search menerima input berupa string hotel search dari user dan output berupa list of hotel_id
Terdapat 5 kondisi inputan:
1. Input user berupa lokasi dengan 3 kata, contoh west valley city
2. Input user berupa lokasi dengan 2 kata, contoh new york
3. Input user berupa lokasi dengan 1 kata, contoh texas
4. Input user berupa spesifik hotel, didapat dengan menquery hasil prediksi rekomendasi hotel FYP yang mengandung kata spesifik
5. Input tidak valid, akan return None/Null

In [3]:
# load user, hotel, and location data
df_user = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/Final_Dataset_User_ML.csv', index_col=[0])
df_location = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Hotel%20Location/location_data.csv', index_col=[0])
df_hotel_nonML = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/Final_Dataset_Hotel_nonML.csv', index_col=[0])

In [37]:
# create location dictionary for fastest data retrieval
dict_location =  df_location.set_index('location_name').T.to_dict('list')
dict_location

{'new york': [40.6943, -73.9249],
 'california': [36.90308075689545, -120.17811173829377],
 'illinois': [40.367719710544456, -89.05563025499656],
 'florida': [28.150635987261147, -82.13648099787684],
 'texas': [30.80193484848485, -97.79111851851852],
 'pennsylvania': [40.60004346590909, -77.62797960227273],
 'georgia': [32.90860421052631, -83.55564255639098],
 'district of columbia': [38.9047, -77.0163],
 'massachusetts': [42.06705905511812, -71.11909370078737],
 'arizona': [33.904746712018145, -111.57759886621317],
 'michigan': [43.57989364864865, -84.98434716216215],
 'washington': [37.96935555555556, -90.1078222222222],
 'minnesota': [45.49070698447894, -94.27401507760533],
 'colorado': [39.18949807692307, -105.2927297008547],
 'maryland': [39.08983625954198, -76.96865],
 'nevada': [38.78712419354838, -117.40238064516129],
 'missouri': [38.45824756097561, -92.4146638836773],
 'oregon': [41.64425, -89.325],
 'puerto rico': [18.24016845878136, -66.38615089605736],
 'ohio': [40.3579689

### Eksperimen 

In [136]:
# %%time
# # filtering hotel berdasarkan hotel_name mengandung kata tertentu
# df_hotel_nonML[df_hotel_nonML['hotel_name'].str.contains("Four Points")].head(3)

CPU times: total: 0 ns
Wall time: 2 ms


,hotel_id,hotel_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,site_review_rating,hotel_facilities,room_facilities
41,H000042,Holiday On Texoma Motel,The standard check-in time is 12:00 PM and the...,575 Highport Rd,"['Chhatrapati Shivaji International Airport', ...",Pottsboro,TX,US,33.8,-96.7,Service Apartment,1,1.0,"('Doctor on Call', 'Laundry Service Available'...","('Attached Bathroom', 'Bathroom Amenities', 'C..."
42,H000043,Holiday Inn Express Suites Lander,The standard check-in time is 12:00 PM and the...,1002 11th St,"['Muniyara Dolmens', 'Marayoor P.O']",Lander,WY,US,42.8,-108.7,Hotel,2,2.0,"('Car Rental', 'Doctor on Call', 'Doorman', 'E...","('Bathroom Amenities', 'Ceiling Fan', 'Newspap..."
66,H000067,Holiday Inn Express Suites Madison Central,The standard check-in time is 12:00 PM and the...,610 John Nolen Dr,['Others'],Madison,WI,US,43.1,-89.4,Service Apartment,0,1.0,"('Doctor on Call', 'Dry Cleaning', 'Front Desk...","('Bathroom Amenities', 'Desk', 'Dressing Area'..."


In [17]:
# prediction = [  6,  83, 138,  21,  84,   4,  62, 115,  77,  34]
#  # convert list of integer ke dalam format ID, identifier data: (H/U/R)
# def convertListInteger(identifier, list_int):
#     converted = []
#     for n in list_int:
#         converted.append(identifier+'{0:06}'.format(n))
#     return converted

# # example
# convertListInteger('H', prediction)

['H000006',
 'H000083',
 'H000138',
 'H000021',
 'H000084',
 'H000004',
 'H000062',
 'H000115',
 'H000077',
 'H000034']

In [143]:
# %time
# # querying data hotel berdasarkan list of hotel_id, then sort based on the list
# df_hotel_copy = df_hotel_nonML.copy()
# df_hotel_copy = df_hotel_copy[df_hotel_copy['hotel_id'].isin(convertListInteger('H', prediction))]
# df_hotel_copy['sort_cat'] = pd.Categorical(df_hotel_copy['hotel_id'], categories=convertListInteger('H', prediction), ordered=True)
# df_hotel_copy.sort_values('sort_cat', inplace=True)
# df_hotel_copy.reset_index(drop=True, inplace=True)
# # df_hotel_copy[df_hotel_copy['hotel_name'].str.contains("Inn")]['hotel_id'].values
# df_hotel_copy

CPU times: total: 0 ns
Wall time: 0 ns


,hotel_id,hotel_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,site_review_rating,hotel_facilities,room_facilities,sort_cat
0,H000006,Sugar Maple Inn,The standard check-in time is 12:00 PM and the...,8546 Edgemont Rd,"['Rg Baruah Road', 'Nehru Stadium', 'Guwahati ...",Higden,AR,US,35.6,-92.2,Hotel,0,5.0,"('Board Room', 'Business Center', 'Car Rental'...","('Bathroom Amenities', 'Ceiling Fan', 'Mirror'...",H000006
1,H000083,The Parisian Courtyard Inn,The standard check-in time is 12:00 PM and the...,1726 Prytania St,"['Gaffar Market', 'Talkatora Indoor Stadium', ...",New Orleans,LA,US,29.9,-90.1,Hotel,1,4.0,"('Air / Rail Booking', 'Baggage Room', 'Car Re...","('Ceiling Fan', 'Housekeeping', 'International...",H000083
2,H000138,Miles City Hotel Suites,The standard check-in time is 12:00 PM and the...,1720 S Haynes Ave,"['Haridwar', 'Laxman Jhula Area']",Miles City,MT,US,46.4,-105.8,Tent,3,5.0,"('Baggage Room', 'Campfire / Bon Fire', 'Front...","('Attached Bathroom', 'Extra Bed, Towels, Line...",H000138
3,H000021,Best Western Plus South Coast Inn,The standard check-in time is 12:00 PM and the...,5620 Calle Real,['Mallandur'],Goleta,CA,US,34.4,-119.8,Homestay,1,1.0,"('Doctor on Call', 'Gardens', 'Guide / Sightse...","('Attached Bathroom', 'Balcony', 'Bathroom Ame...",H000021
4,H000084,Hotel Indigo Atlanta Downtown,The standard check-in time is 12:00 PM and the...,230 Peachtree St Nw,"['Chapora Fort', 'Fort Aguada', 'Palolem Beach...",Atlanta,GA,US,33.8,-84.4,Hotel,2,4.5,"('Bar / Lounge ', 'Laundry Service Available',...","('Air Conditioning', 'Ensuite / Private Bathro...",H000084
5,H000004,JW Marriott Atlanta Buckhead,The standard check-in time is 12:00 PM and the...,3300 Lenox Road NE,"['International Airport', 'Domestic Airport', ...",Atlanta,GA,US,33.8,-84.4,Hotel,3,4.0,"(' Drivers Rest Room', 'ATM / Banking', 'Adjoi...","('Air Conditioning', 'Bathrobes', 'Breakfast A...",H000004
6,H000062,Motel 6,The standard check-in time is 12:00 PM and the...,2745 S Woodlands Village Blvd,"['Fort Kochi', 'Bolgatty Palace', 'Jewish Syna...",Flagstaff,AZ,US,35.2,-111.7,Homestay,0,5.0,"('Doctor on Call', 'Front Desk', 'Laundry Serv...","('Air Conditioning', 'Attached Bathroom', 'Bat...",H000062
7,H000115,Red Roof Plus+ Phoenix West,The standard check-in time is 12:00 PM and the...,5215 W Willetta St,"['Indian Institute of Astrophysics', 'Koramang...",Phoenix,AZ,US,33.5,-112.2,Lodge,0,1.0,"('Baggage Room', 'Doctor on Call', 'Front Desk...","('Bathroom Amenities', 'Breakfast Available', ...",H000115
8,H000077,Hampton Inn Suites DetroitCanton,The standard check-in time is 11:00 AM and the...,1950 N Haggerty Rd,"['Anjuna Area', 'Anjuna Beach', 'Calangute Bea...",Canton,MI,US,42.3,-83.4,Guest House,0,5.0,"('Laundry Service Available', 'Parking Facility')","('Attached Bathroom', 'Breakfast Available', '...",H000077
9,H000034,Hampton Inn Wilkesboro,The standard check-in time is 12:00 PM and the...,1300 Collegiate Drive,"['Ropeway Lower Station', 'Mall Road', 'Others']",Wilkesboro,NC,US,36.1,-81.2,Tent,3,5.0,"('Doctor on Call',)","('Breakfast Available', 'Housekeeping')",H000034


In [165]:
# df_hotel_nonML['hotel_name'].apply(cleanString)

0       homewood suites by hilton raleighdurham aprese...
1       four points by sheraton orlando international ...
2                     hampton inn pittsburgh west mifflin
3                            jw marriott atlanta buckhead
4                                    best western rambler
                              ...                        
1306                                        cupertino inn
1307                 best western plus concordville hotel
1308                 georgia tech hotel conference center
1309            best western plus nashville airport hotel
1310    best western plus dallas hotel   conference ce...
Name: hotel_name, Length: 1311, dtype: object

### Helper Functions 

In [110]:
# konversi input kata menjadi list n-gram 
def convertStringToNgram(n, input):
    return list(ngrams(input,n))

# example
convertStringToNgram(2, 'new york hotel'.split())

[('new', 'york'), ('york', 'hotel')]

In [164]:
# function untuk membersihkan input string
def cleanString(input):
    # remove symbol (get alphanumeric only)
    input = re.sub(r'[^\w]', ' ', input)
    # lowercase the input
    input = input.lower()
    return input

### Main Hotel Search Function 

In [173]:
import re
# test input

# input = '#@West valley City' # skenario 3 kata
# input = 'Hotel @#$New !York' # skenario 2 kata
# input = '@#$Hotel !Texas ' # skenario 1 kata
# input = '@#$Inn ' # skenario spesifik hotel
input = '@#$asdasd ' # skenario spesifik hotel tidak valid


# take a user data as an example
user = df_user.iloc[0]

In [174]:
%%time
# input berupa string search user, data user, dan model
# output berupa list hotel yang akan ditampilkan
def HotelSearch(input, user, df_hotel):
#     menggunakan contoh hasil prediksi model berjumlah 10 berikut
#     prediction = [  6,  83, 138,  21,  84,   4,  62, 115,  77,  34]
    # clean input string
    input = cleanString(input)
    # split input into words
    input = input.split()
    
    # initiate df_h a copy of df_hotel
    df_h = df_hotel.copy()
    df_h['hotel_name'] = df_h['hotel_name'].apply(cleanString)
    
    model_input_vector = user.iloc[3:].copy()
    # inisiasi boolean jika lokasi telah ditemukan
    loc_found = False
    
    # pengecekan untuk lokasi yang terdiri dari 3 kata
    if len(input) >= 3 :
        # mengkonversi input user menjadi pasangan kata n-gram
        for item in convertStringToNgram(3, input):  
            ngram_word = ' '.join(item)
            if ngram_word in dict_location:
                loc = dict_location.get(ngram_word)
                loc_found = True
                print('3 kata sukses')
    # pengecekan untuk lokasi yang terdiri dari 2 kata
    if not loc_found:
        for item in convertStringToNgram(2, input):
            ngram_word = ' '.join(item)
            if ngram_word in dict_location:
                loc = dict_location.get(ngram_word)
                loc_found = True
                print('2 kata sukses')
    # pengecekan untuk lokasi yang terdiri dari 1 kata
    if not loc_found:
        for word in input:
            # jika input berupa nama lokasi daerah akan mengganti lokasi koordinat user
            if word in dict_location:
                loc = dict_location.get(word)
                loc_found = True
                print('1 kata sukses')

# pengecekan ditemukan lokasi atau tidak
    if loc_found:
        model_input_vector[['latitude','longitude']] = loc
        # melakukan prediksi
        # prediction = model.predict(model_input_vector)
        prediction = [  6,  83, 138,  21,  84,   4,  62, 115,  77,  34]
        output = convertListInteger('H', prediction)
    else:
    # menggunakan hasil predict untuk hotel FYP, namun pada code ini akan dilakukan prediksi ulang
        # melakukan prediksi
        # prediction = model.predict(model_input_vector)
        prediction = [  6,  83, 138,  21,  84,   4,  62, 115,  77,  34]
        # code dibawah akan mengcopy df hotel, kemudian menquery berdasarkan hotel_id hasil prediksi
        # kemudian dilakukan sorting berdasarkan urutan dari list hasil prediksi
        # kemudian dilakukan filter berdasarkan hotel_name yang mengandung kata input
        for word in input:
            df_h = df_h[df_h['hotel_id'].isin(convertListInteger('H', prediction))]
            df_h['sort'] = pd.Categorical(df_h['hotel_id'], categories=convertListInteger('H', prediction), ordered=True)
            df_h.sort_values('sort', inplace=True)
            df_h.reset_index(drop=True, inplace=True)
            arr_hotel_id = df_h[df_h['hotel_name'].str.contains(word)]['hotel_id'].values
            if len(arr_hotel_id) != 0:
                output = arr_hotel_id
                print('kata spesifik sukses')
                break
            else:
                output = None
                print('kata spesifik tidak valid sukses')
    return output
    
HotelSearch(input, user, df_hotel_nonML)

kata spesifik tidak valid sukses
CPU times: total: 0 ns
Wall time: 5.01 ms
